## Imports

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath('..'))

import cpso
import optimizationFuncs as optim
import pso_util as pu

## Reload (if necessary)

In [ ]:
from importlib import reload
cpso = reload(cpso)
optim = reload(optim)
pu = reload(pu)

## Setup

In [ ]:
func = optim.schaffer4
funckey = lambda x : func(x.reshape(1, -1))[0]
Np = 25
D = 2
llim = np.repeat(-5.12, D)
rlim = np.repeat(5.12, D)
mizer = cpso.RI_PSO(func, llim, rlim, Np)

## FRF Loop

In [ ]:
opts = []

opt, _ = mizer.forward()
minopt = opt
print(f"Optimum = {opt}\n")
opts.append(opt)


for _ in range(3) :
    opt = opts[-1]
    mizer.reverse(opt)
    new_opt, _ = mizer.forward()
    if funckey(new_opt) - funckey(minopt) >= 1e-1 :
        break
    else :
        print(f"Optimum = {new_opt}\n")
        if funckey(new_opt) < funckey(minopt) :
            minopt = new_opt
        opts.append(new_opt)

        
opts = np.array(opts)
print(f"Optimums = {opts}")

## Plot 1st optima, hull and 2nd optima

In [ ]:
hull, pip, hb = mizer.hulls[0]

fig, ax = plt.subplots(1, 1)
fig.set_figheight(7.5)
fig.set_figwidth(7.5)

ax.plot(pip[0], pip[1], 'ro', label='global optima')
ax.plot(hb[:,0], hb[:,1], 'ko', label='reverse-PSO final position')
for i,j in zip(hull.vertices[:-1], hull.vertices[1:]) :
    x = [hb[i,0], hb[j,0]]
    y = [hb[i,1], hb[j,1]]
    ax.plot(x, y, 'b')
    
i = hull.vertices[-1]
j = hull.vertices[0]
x = [hb[i,0], hb[j,0]]
y = [hb[i,1], hb[j,1]]
ax.plot(x, y, 'b')


ax.plot(mizer.particles[:,0], mizer.particles[:,1], 'go', markersize=5, label='other optima')
ax.set_title('Schaffer-4')
ax.set_xlim([-2, 2])
ax.set_ylim([-2, 2])
ax.grid('on')
ax.legend(loc='lower left')

pass